In [ ]:
%pip install --quiet langchain openai langchain-openai neo4j tqdm unstructured

In [ ]:
%pip install python-dotenv

In [1]:
from langchain.graphs.neo4j_graph import Neo4jGraph
import os
from dotenv import load_dotenv

load_dotenv()
# url="bolt://18.234.191.206:7687",
# username= "neo4j",
# password="grasses-extenuations-moments"
# url="bolt://44.202.63.193:7687",
# username= "neo4j",
# password="string-cheaters-combatants"

# os.environ["NEO4J_URI"] = "bolt://44.202.63.193:7687"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] ="string-cheaters-combatants"

url=os.getenv('NEO4J_URI'),
username= os.getenv('NEO4J_USERNAME'),
password=os.getenv('NEO4J_PASSWORD')


os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = os.getenv('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] =os.getenv('NEO4J_PASSWORD')


graph = Neo4jGraph()

In [2]:
%pip install wikipedia-api

Note: you may need to restart the kernel to use updated packages.


In [3]:

from langchain.pydantic_v1 import Field, BaseModel
from models.graph_entities.node import Node
from models.graph_entities.relationship import Relationship
from models.graph_entities.knowledge_graph import KnowledgeGraph
from models.graph_entities.property import Property
from typing import List, Optional
from langchain.graphs.graph_document import Node as BaseNode
from langchain.pydantic_v1 import Field
from typing import List, Optional
from langchain.graphs.graph_document import Relationship as BaseRelationship

from langchain.pydantic_v1 import Field
from typing import List
from langchain.pydantic_v1 import Field, BaseModel


In [4]:

#from node import Node
#from relationship import Relationship
from langchain.graphs.graph_document import Node as BaseNode
from langchain.graphs.graph_document import Relationship as BaseRelationship

In [6]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )

def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

In [7]:
from typing import List, Optional
from langchain.chains.openai_functions import (
    create_structured_output_runnable,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

#from knowledge_graph import KnowledgeGraph

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)


def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    compulsory_node_attributes: Optional[List[str]] = None,
    allowed_relationships: Optional[List[str]] = None,
):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph, which will help in fetching movies related information.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes and Attributes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"** .or try to be more specific terms like "actor" or "director".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
- **Allowed Node Labels and their Compulsory Attributes**:
  - Node Label: Person
      - Compulsory Attributes of a Person: Name, Role, Date of Birth.
  - Node Label: Object
      - Compulsory Attributes of an Object: Name, Role, Location.
  - Node Label: Location
      - Compulsory Attributes of a Location: Name, Address, City, State, Country.
  - Node Label: Event
      - Compulsory Attributes of an Event: Date, Time, Location.
- **Attribute Values**:
  - Use N/A when the value for a compulsory attribute is missing.
  - The Role of Person Nodes and Object Nodes must clearly convey their overall role in the events taking place, such as Hero, Villan.
  - The Name of a Location must be human readable, and comprehendible, such as, Walmart.
  - Do not Name events as just a date, time, or the name of the day, or an incrementing Id, such as Event1, Event2, 27/05/2015, or Saturday.
  - Name of Events must be meaningful, for example, Theft, Chase, 911 Call, Murder, etc.
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.""",
            ),
            (
                "human",
                "Use the given format to extract information from the following input: {input}",
            ),
            (
                "human",
                "Tip: Make sure to answer in the correct format",
            ),
        ]
    )
    # return create_structured_output_runnable(KnowledgeGraph, llm, prompt, verbose=False)
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)



# def get_extraction_chain(
#     allowed_nodes: Optional[List[str]] = None,
#     compulsory_node_attributes: Optional[List[str]] = None,
#     allowed_relationships: Optional[List[str]] = None,
# ):
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#             f"""# Advanced Knowledge Graph Extraction Instructions for GPT-4
# ## 1. Overview
# You are tasked with extracting structured information to build a detailed knowledge graph for movie-related content. This includes identifying complex relationships and key events.
# ## 2. Detailed Instructions on Entities and Relationships
# - **Entities**: Extract entities such as persons, objects, and locations with high precision. For each entity, identify relevant attributes.
# - **Complex Relationships**: Focus on extracting detailed relationships. For example:
#   - Familial ties (e.g., Walter White is the husband of Skyler White).
#   - Professional relationships and hierarchies.
#   - Significant historical or plot-defining interactions (e.g., purchase of items, key decisions).
# - **Events**: Extract and describe significant events in detail, specifying the entities involved, the nature of the event, and its impact on the plot or character development.
# ## 3. Format and Compliance
# - Ensure all data is in key-value format with attributes clearly defined and relationships explicitly stated.
# - Use contextual cues to infer relationships and events not explicitly mentioned but implied by the narrative context.
# ## 4. Examples
# - If Walter and Skyler White are mentioned in different contexts but their interaction implies a marital relationship, this should be explicitly extracted and noted.
# - Extract events like 'Walter White buys an SUV' and note its significance in the plot, such as 'used for cooking methamphetamine'.
# ## 5. Coreference and Consistency
# - Maintain strict consistency in entity references. Use the most complete and contextually appropriate identifier for entities throughout the knowledge graph.
# ## 6. Feedback
# - Your output will be reviewed for precision in entity relationship extraction and event detailing. Strive for accuracy and depth in your responses."""
#        ),
#             (
#                 "human",
#                 "Use the given format to extract information from the following input: {input}",
#             ),
#             (
#                 "human",
#                 "Tip: Make sure to answer in the correct format",
#             ),
#         ]
#     )
#     # return create_structured_output_runnable(KnowledgeGraph, llm, prompt, verbose=False)
#     return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [8]:
from langchain_core.documents import Document
from langchain.graphs.graph_document import GraphDocument


def extract_and_store_graph(
    document: Document,
    nodes: Optional[List[str]] = None,
    node_attributes: Optional[List[str]] = None,
    relationships: Optional[List[str]] = None,
) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, node_attributes, relationships)
    data = extract_chain.invoke(document.page_content)["function"]
    # Construct a graph document
    graph_document = GraphDocument(
        nodes=[map_to_base_node(node) for node in data.nodes],
        relationships=[map_to_base_relationship(rel) for rel in data.relationships],
        source=document,
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [9]:
from typing import List
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_core.documents.base import Document


def extract_documents_from_directory(path: str) -> List[Document]:
    # Read the text document
    raw_documents = DirectoryLoader(path).load()
    # Define chunking strategy
    text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

    # Only take the first the raw_documents
    documents = text_splitter.split_documents(raw_documents)
    return documents


def extract_documents_from_file(path: str) -> List[Document]:
    # Read the text document
    raw_documents = TextLoader(path).load()
    # Define chunking strategy
    text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

    # Only take the first the raw_documents
    documents = text_splitter.split_documents(raw_documents)
    return documents



def extract_documents_from_text(text: str) -> List[Document]:
    """Split a given text into smaller parts based on a defined chunk size and overlap.

    Args:
        text (str): The input text to split.

    Returns:
        List[Document]: A list of Document objects.
    """
    # Define chunking strategy
    text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

    # Convert the text to a Document object
    document = Document(page_content=text)

    # Split the document into smaller documents
    documents = text_splitter.split_documents([document])

    return documents

In [10]:
from tqdm import tqdm

# Specify which node labels should be extracted by the LLM
allowed_nodes = ["Person", "Object", "Location", "Event"]
compulsory_node_attributes = {
    "Person": ["Name", "Role", "DateOfBirth"],
    "Object": ["Name", "Role", "Location"],
    "Location": ["Name", "City", "State", "Country"],
    "Event": ["Date", "Time", "Location"]
}

def plot_directory_data(path: str):
    documents = extract_documents_from_directory(path)
    for i, d in tqdm(enumerate(documents), total=len(documents)):
        extract_and_store_graph(d, allowed_nodes, compulsory_node_attributes)


def plot_file_data(path: str):
    documents = extract_documents_from_file(path)
    for i, d in tqdm(enumerate(documents), total=len(documents)):
        extract_and_store_graph(d, allowed_nodes, compulsory_node_attributes)


def plot_text_data(text: str):
    documents = extract_documents_from_text(text)
    for i, d in tqdm(enumerate(documents), total=len(documents)):
        extract_and_store_graph(d, allowed_nodes, compulsory_node_attributes)

In [11]:
import requests
from bs4 import BeautifulSoup

def fetch_wikipedia_content(url):
    """Fetch the content of a Wikipedia page given its URL."""
    response = requests.get(url)
    if response.status_code != 200:
        return None
    return response.text

def extract_relevant_text(html_content):
    """Extract relevant sections like the summary and plot from Wikipedia HTML content."""
    soup = BeautifulSoup(html_content, 'html.parser')
    # Assuming the summary is in the first paragraph of the page
    summary = soup.find('p').get_text()
    # Extract other relevant sections if necessary
    return summary

def plot_kg_from_wikipediaa(url):
    """Fetch and extract content from Wikipedia and use it to plot the knowledge graph."""
    html_content = fetch_wikipedia_content(url)
    if html_content:
        text = extract_relevant_text(html_content)
        print("entered")
        text
        # Assuming 'plot_text_data' is a function you have that can take text and plot the KG
        #plot_text_data(text)  # This function needs to be defined or adjusted according to your setup
    else:
        print("Failed to fetch content from Wikipedia")

In [12]:
# Import packages
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import wikipediaapi
def plot_kg_from_wikipedia(urll):
# Specify url of the web page
  source = urlopen(urll).read()

  # Make a soup
  soup = BeautifulSoup(source,'lxml')
  soup

  # Extract the plain text content from paragraphs
  paras = []
  for paragraph in soup.find_all('p'):
      paras.append(str(paragraph.text))

  # Extract text from paragraph headers
  heads = []
  for head in soup.find_all('span', attrs={'mw-headline'}):
      heads.append(str(head.text))

  # Interleave paragraphs & headers
  text = [val for pair in zip(paras, heads) for val in pair]
  text = ' '.join(text)

  # Drop footnote superscripts in brackets
  text = re.sub(r"\[.*?\]+", '', text)

  # Replace '\n' (a new line) with '' and end the string at $1000.
  text = text.replace('\n', '')[:-11]
  #print(text)
  plot_text_data(text)
  return text


def plot_from_wikipedia(title):

  wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')

  page_py = wiki_wiki.page(title)
  print(page_py.fullurl)
  data = page_py.text
  #print(data)
  plot_text_data(data)
  return data



#plot_from_wikipedia('Breaking Bad')
#plot_kg_from_wikipedia('https://en.wikipedia.org/wiki/Breaking_Bad')


In [13]:
#plot_file_data("Breaking_Bad.txt")
#plot_kg_from_wikipedia('https://en.wikipedia.org/wiki/Breaking_Bad') #by link
plot_from_wikipedia('El Camino: A Breaking Bad Movie') # by wikipedia title


https://en.wikipedia.org/wiki/El_Camino:_A_Breaking_Bad_Movie


  0%|          | 0/6 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_structured_output_runnable instead.
  warn_deprecated(
 83%|████████▎ | 5/6 [11:31<02:18, 138.35s/it]


ValidationError: 1 validation error for _OutputFormatter
__root__
  Unterminated string starting at: line 3080 column 13 (char 59974) (type=value_error.jsondecode; msg=Unterminated string starting at; doc={
  "output": {
    "nodes": [
      {
        "id": "Vince Gilligan",
        "type": "person",
        "properties": [
          {
            "key": "name",
            "value": "Vince Gilligan"
          }
        ]
      },
      {
        "id": "Breaking Bad",
        "type": "object",
        "properties": [
          {
            "key": "name",
            "value": "Breaking Bad"
          }
        ]
      },
      {
        "id": "Better Call Saul",
        "type": "object",
        "properties": [
          {
            "key": "name",
            "value": "Better Call Saul"
          }
        ]
      },
      {
        "id": "El Camino",
        "type": "object",
        "properties": [
          {
            "key": "name",
            "value": "El Camino"
          }
 

In [ ]:
# Query the knowledge graph in a RAG application
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

In [ ]:
cypher_chain.invoke({"query": "who is Walter White married to?"})

In [ ]:
#!pip install selenium
!pip install wikipedia-api

In [ ]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')

page_py = wiki_wiki.page('El Camino: A Breaking Bad Movie')

print("Page - Exists: %s" % page_py.exists())
# Page - Exists: True

print("Page - Title: %s" % page_py.title)
# Page - Title: Python (programming language)

#print("Page - Summary: %s" % (page_py.summary))
print(page_py.fullurl)

#p_wiki = wiki_wiki.page("Test 1")
print(len(page_py.text))

